<a href="https://colab.research.google.com/github/GuillermoPerez2905/Audio-Recognition-Program/blob/main/Matrix_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORTACIÓN INSTALACIÓN DE PAQUETES NECESARIOS**

In [ ]:
!pip install python_speech_features
!pip install -U -q PyDrive

In [ ]:
from python_speech_features import mfcc
from python_speech_features import logfbank
from python_speech_features import delta
import scipy.io.wavfile as wav
import pandas as pd
import numpy as np
import matplotlib.pyplot as pltn
import seaborn as sns
import pandas.util.testing as tm
import glob
import statistics as stats
%matplotlib inline
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

**CÓDIGO PARA GENERAR LA MATRIZ DE COEFICIENTES**

*MFCC'S DE TODOS LOS AUDIOS*

In [ ]:
MFCC_values = pd.DataFrame()
speaker_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
                '11', '12', '13','14', '15', '16', '17', '18', '19', '20',
                '21', '22', '23', '24', '25']

audio_number = ['00','01', '02', '03', '04', '05', '06', '07', '08', '09', '10']
for speaker in speaker_list: #Bucle para acceder a todos los audios del Dataset
  for method in ['M', 'm', 'T', 't']:
    for ind in np.arange(0,11): 
        for letter in ['A', 'B', 'C', 'D', 'E', 'F']:
          for audio in audio_number:  
            try:
              wavfile = '0' + speaker + method + str(ind)+ letter + audio
              #Sacamos la frecuencia de muestreo del audio y los datos referentes a la señal
              (rate,sig) = wav.read("/content/drive/My Drive/Audios Ahumada/" + wavfile +".wav")
              #Sacamos los MFCC de esa señal y frecuencia y los añadimos al Dataframe
              #cada audio que vaya introduciéndose al bucle 
              MFCC_values = MFCC_values.append(pd.DataFrame(mfcc(sig,rate)))
            except FileNotFoundError:
              continue
              
MFCC_values



In [ ]:
MFCC_values = pd.DataFrame()
#Uso de la función glob para obtener los audios sin necesidad de usar bucles
lista_audios = sorted(glob.glob("/content/drive/My Drive/Audios Ahumada/*wav")) 
for audio in lista_audios:
  (rate,sig) = wav.read(audio)
  name = audio.split('/')[5].split('.')[0]
  if 'm' in name.lower():
    name = name.lower().split('m')[0]
  elif 't' in name.lower():
    name= name.lower().split('t')[0]
  mfcc_audio = mfcc(sig,rate)
  MFCC_value = pd.DataFrame(mfcc_audio, index=int(name)*np.ones(mfcc_audio.shape[0]))
  MFCC_values = MFCC_values.append(MFCC_value)
MFCC_values

*VALORES MEDIOS DE LOS MFCC*

In [ ]:
MFCC_mean_values = pd.DataFrame()
lista_audios = sorted(glob.glob("/content/drive/My Drive/Audios Ahumada/*wav"))
for audio in lista_audios:
  (rate,sig) = wav.read(audio)
  print('Voy por ' + audio.split('/')[5])
  MFCC_single_mean = pd.DataFrame(np.apply_along_axis(
      stats.mean, 0, mfcc(sig,rate)), columns= [audio.split('/')[5]],
       index= pd.RangeIndex(1,14,1)).transpose()
  MFCC_mean_values = MFCC_mean_values.append(MFCC_single_mean)
MFCC_mean_values

*VALORES MEDIOS MÁS LAS DERIVADAS DE LOS MFCC*

In [ ]:
MFCC_m_d_values = pd.DataFrame()
MFCC_derivated_values = pd.DataFrame()
lista_audios = sorted(glob.glob("/content/drive/My Drive/Audios Ahumada/*wav"))
for audio in lista_audios:
  (rate,sig) = wav.read(audio)
  print('Voy por ' + audio.split('/')[5])
  MFCC_single_mean = pd.DataFrame(np.apply_along_axis(
      stats.mean, 0, mfcc(sig,rate, winstep= 0.025)), index= pd.RangeIndex(1,14,1) ,columns= [audio.split('/')[5]]).transpose()
  MFCC_derivate = pd.DataFrame(np.apply_along_axis(
      stats.mean , 0, delta(mfcc(sig,rate, winstep= 0.025),165j)),index= pd.RangeIndex(1,14,1), columns= [audio.split('/')[5]]).transpose()
  MFCC_m_d_values = MFCC_m_d_values.append(MFCC_single_mean)
  MFCC_derivated_values = MFCC_derivated_values.append(MFCC_derivate)
MFCC_m_d_values = MFCC_m_d_values.join(MFCC_derivated_values,
                                         how= 'right',lsuffix='_mean', rsuffix= '_deriv')
MFCC_m_d_values

*VALORES MEDIOS MÁS LA VARIANZA DE LOS MFCC*

In [ ]:
MFCC_m_s_values = pd.DataFrame()
MFCC_variance_values = pd.DataFrame()
lista_audios = sorted(glob.glob("/content/drive/My Drive/Audios Ahumada/*wav"))
for audio in lista_audios:
  (rate,sig) = wav.read(audio)
  print('Voy por ' + audio.split('/')[5])
  MFCC_single_mean = pd.DataFrame(np.apply_along_axis(
      stats.mean, 0, mfcc(sig,rate, winstep= 0.025)), index= pd.RangeIndex(1,14,1) ,columns= [audio.split('/')[5]]).transpose()
  MFCC_variance = pd.DataFrame(np.apply_along_axis(
      stats.variance, 0, mfcc(sig,rate, winstep= 0.025)),index= pd.RangeIndex(1,14,1), columns= [audio.split('/')[5]]).transpose()
  MFCC_m_s_values = MFCC_m_s_values.append(MFCC_single_mean)
  MFCC_variance_values = MFCC_variance_values.append(MFCC_variance)
  #if audio.split('/')[5] == '001M2C01.wav':
    #break
MFCC_m_s_values = MFCC_m_s_values.join(MFCC_variance_values,
                                         how= 'right',lsuffix='_mean', rsuffix= '_var')
MFCC_m_s_values

**GUARDAR AUDIOS EN GOOGLE DRIVE**

In [ ]:
MFCC_values.to_csv('000_MFCC_values.csv')
!cp 000_MFCC_values.csv "/content/drive/My Drive/Audios Ahumada"

In [ ]:
MFCC_mean_values.to_csv('000_MFCC_mean_values.csv')
!cp 000_MFCC_mean_values.csv "/content/drive/My Drive/Audios Ahumada"

In [ ]:
MFCC_m_d_values.to_csv('000_MFCC_m_d_values.csv')
!cp 000_MFCC_m_d_values.csv "/content/drive/My Drive/Audios Ahumada"

In [ ]:
MFCC_m_s_values.to_csv('000_MFCC_m_s_values.csv')
!cp 000_MFCC_m_s_values.csv "/content/drive/My Drive/Audios Ahumada"